In [2]:
#Siyuan Li
#last update: 2024/2/1


from sklearn.linear_model import LassoCV, ElasticNetCV,LogisticRegression
from sklearn.model_selection import cross_val_predict,train_test_split,KFold
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor


import pandas as pd
import os

In [21]:
data = pd.read_csv("E:/ewas/colect/GSE48684_Matrix_signal_intensities_2.txt",sep="\t", header=0)  


data.head()



C:\Users\LSY\AppData\Local\Temp\ipykernel_28616\2937087123.py:1: DtypeWarning: Columns (10,14,22,42,46,90) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("E:/ewas/colect/GSE48684_Matrix_signal_intensities_2.txt",sep="\t", header=0)


,Index,TargetID,16748.AVG_Beta,16748.Signal_A,16748.Signal_B,16748.Detection Pval,16740.AVG_Beta,16740.Signal_A,16740.Signal_B,16740.Detection Pval,...,1186A.Signal_B,1186A.Detection Pval,7497.AVG_Beta,7497.Signal_A,7497.Signal_B,7497.Detection Pval,2026.AVG_Beta,2026.Signal_A,2026.Signal_B,2026.Detection Pval
0,1,cg00000029,0.2441185,2374.0,799.0,0.0,0.210745,2559.0,710.0,0.0,...,1038.0,0.0,0.2300718,1938.0,609.0,0.0,0.2839292,1963.0,818.0,0.0
1,2,cg00000108,0.9686329,260.0,11117.0,0.0,0.9489362,464.0,10481.0,0.0,...,10965.0,0.0,0.9562619,449.0,12003.0,0.0,0.9604881,308.0,9918.0,0.0
2,3,cg00000109,0.8947664,461.0,4770.0,0.0,0.8844879,562.0,5069.0,0.0,...,5380.0,0.0,0.9217288,427.0,6206.0,0.0,0.9259907,408.0,6356.0,0.0
3,4,cg00000165,0.3461618,3043.0,1664.0,0.0,0.2746345,3323.0,1296.0,0.0,...,2456.0,0.0,0.5740005,1509.0,2168.0,0.0,0.779024,1023.0,3959.0,0.0
4,5,cg00000236,0.8537285,818.0,5358.0,0.0,0.8941941,649.0,6330.0,0.0,...,5772.0,0.0,0.8995675,527.0,5616.0,0.0,0.8821441,597.0,5217.0,0.0


In [29]:

CG = ['cg06551493', 'cg01419670', 'cg16530981', 'cg18022036','cg12691488', 'cg17292758', 'cg16170495',
      'cg11240062', 'cg21585512', 'cg24702253', 'cg17187762', 'cg05983326', 'cg06825163',
      'cg11885357', 'cg08829299', 'cg07044115']
beta= [-0.41, 0.4332, 0.2895, -0.5172, -0.3915, -0.3246, -0.2886, 0.2451, -0.5651, 
             0.3615, -0.2445, -0.3951, -0.5089, -0.2504, -0.2357,-0.3607]

result_data = data[data['TargetID'].isin(CG)]


result_data = result_data.set_index('TargetID').loc[CG].reset_index()
result_data = result_data[result_data.columns[result_data.columns.str.endswith('Beta')].insert(0, 'TargetID')]

display(result_data)

,TargetID,16748.AVG_Beta,16740.AVG_Beta,16743.AVG_Beta,16725.AVG_Beta,16730.AVG_Beta,16749.AVG_Beta,16733.AVG_Beta,16736.AVG_Beta,16747.AVG_Beta,...,16745.AVG_Beta,16750.AVG_Beta,TY1226B.AVG_Beta,TY0652B.AVG_Beta,TY1374A.AVG_Beta,TY0905A.AVG_Beta,1902.AVG_Beta,1186A.AVG_Beta,7497.AVG_Beta,2026.AVG_Beta
0,cg06551493,0.06965552,0.07852994,0.08501779,0.09878748,0.09191686,0.1190716,0.0653009,0.09049923,0.09949208,...,0.08401335,0.09685321,0.06005111,0.06972774,0.05122403,0.05414607,0.04579316,0.05805704,0.04476644,0.05778398
1,cg01419670,0.8184508,0.7863777,0.8014854,0.838587,0.8150235,0.7993745,0.848408,0.8381503,0.6660274,...,0.8346739,0.8380827,0.7584763,0.7382449,0.8280247,0.8742244,0.8350266,0.8440517,0.8336567,0.7984232
2,cg16530981,0.4538745,0.6205934,0.4951347,0.5752736,0.7043236,0.5516053,0.6586487,0.5234771,0.5173587,...,0.6234474,0.6155576,0.6468385,0.681998,0.7534153,0.6606437,0.7160904,0.7863613,0.7499441,0.7925668
3,cg18022036,0.6253619,0.5493444,0.6315137,0.511761,0.6002338,0.7158621,0.6684456,0.5270967,0.4458525,...,0.6654602,0.5591225,0.7158411,0.71652,0.5739051,0.568286,0.6261755,0.5284768,0.5084746,0.5444215
4,cg12691488,0.6799617,0.4337809,0.5506924,0.6060073,0.6519858,0.7982851,0.5242471,0.6660689,0.6695281,...,0.6373314,0.2965591,0.6736009,0.4332798,0.4123005,0.521518,0.6735446,0.4646483,0.772557,0.7883914
5,cg17292758,0.6616608,0.8039936,0.7929771,0.7985718,0.7836755,0.8408464,0.6260213,0.8102,0.7885426,...,0.8564081,0.8448089,0.7635987,0.8005666,0.7391173,0.5357578,0.8328986,0.2875924,0.8596707,0.832213
6,cg16170495,0.1069959,0.09912477,0.1249336,0.1069731,0.1144451,0.06133094,0.07483444,0.067301,0.1089128,...,0.05705641,0.1293916,0.08036391,0.06230785,0.1105548,0.06683003,0.09309839,0.1486245,0.07629912,0.09452479
7,cg11240062,0.2161389,0.1926048,0.2249431,0.2071861,0.5187703,0.03781746,0.7675002,0.4544789,0.7344431,...,0.469722,0.07300885,0.6713117,0.4177045,0.363598,0.403968,0.1101145,0.1113536,0.3953167,0.06004619
8,cg21585512,0.1243465,0.1448887,0.1717589,0.1165579,0.2095688,0.1227533,0.1363256,0.1038292,0.1189682,...,0.1145553,0.1843779,0.1687784,0.1888481,0.2338114,0.1264281,0.1064739,0.1729414,0.09689128,0.1039803
9,cg24702253,0.6054264,0.6659177,0.657255,0.6996252,0.7424939,0.6007583,0.4207692,0.7174292,0.6371604,...,0.6809697,0.6655966,0.6647593,0.5540618,0.622335,0.6671193,0.7190582,0.6444454,0.771656,0.4447241


In [38]:

result_data.iloc[:, 1:] = result_data.iloc[:, 1:].replace(" ", np.nan).apply(pd.to_numeric)
result_matrix = np.matmul(beta, result_data.iloc[:, 1:].values)

print("矩阵乘法结果：", result_matrix)



矩阵乘法结果： [-0.95265140591 -1.014050694856 -1.1665699803539997 -0.8193488427850002
 -1.062980687822 -1.260714424564 -0.7425412036980001 -0.7867967476150001
 -0.8502777261299999 -1.0820171794100002 -1.0747323772400001
 -1.2353653638970001 -1.04106429952 nan -1.005475082776
 -1.0933597888450002 -0.9927363235789999 -1.058663816563 -0.998907726073
 -0.786692037596 -1.038433507314 -0.7421161313270002 -0.879700318994
 -1.192779810139]


The MRS (range, − 5.59 to 4.35) was significantly higher for CRC subjects than in healthy normal subjects (P <  0.000), with a median MRS of 1.68 (IQR, 1.43) in CRC subjects and − 0.430 (IQR, 2.89) in healthy normal subjects.
The MRS showed a good predictive ability for discriminating between CRC and healthy normal subjects (AUC, 0.85; 95% CI: 0.81, 0.89).